In [ ]:
import gradio as gr
import cv2
import numpy as np

# Function to convert 2x3 affine matrix to 3x3 for matrix multiplication
def to_3x3(affine_matrix):
    return np.vstack([affine_matrix, [0, 0, 1]])

# Function to apply transformations based on user inputs
def apply_transform(image, scale, rotation, translation_x, translation_y, flip_horizontal):

    # 将 PIL 格式的图像转换为 NumPy 数组
    image = np.array(image)
    
    # 为了避免边界问题，对图像进行填充
    pad_size = min(image.shape[0], image.shape[1]) // 2
    image_new = np.zeros((pad_size*2+image.shape[0], pad_size*2+image.shape[1], 3), dtype=np.uint8) + np.array((255,255,255), dtype=np.uint8).reshape(1,1,3)
    image_new[pad_size:pad_size+image.shape[0], pad_size:pad_size+image.shape[1]] = image
    image = np.array(image_new)
    
    # 初始化变换后的图像
    transformed_image = np.array(image)
    
    ### 实现复合变换
    # 注意：对于缩放和旋转，需要围绕图像中心进行

    # 获取图像中心坐标
    center = (image.shape[1] / 2, image.shape[0] / 2)
    
    # 生成旋转和缩放矩阵（2x3），围绕图像中心
    rot_mat = cv2.getRotationMatrix2D(center, rotation, scale)
    # 将 2x3 的矩阵转换为 3x3，用于矩阵乘法
    rot_mat_3x3 = to_3x3(rot_mat)
    
    # 生成平移矩阵（3x3）
    trans_mat = np.array([
        [1, 0, translation_x],
        [0, 1, translation_y],
        [0, 0, 1]
    ], dtype=np.float32)
    
    # 生成水平翻转矩阵（3x3）
    if flip_horizontal:
        flip_mat = np.array([
            [-1, 0, image.shape[1]],  # 围绕垂直轴进行翻转
            [0, 1, 0],
            [0, 0, 1]
        ], dtype=np.float32)
    else:
        flip_mat = np.eye(3, dtype=np.float32)  # 单位矩阵（不进行翻转）
    
    # 组合所有的变换矩阵
    # 注意矩阵乘法的顺序，右乘表示先进行的变换
    # 变换顺序：先翻转 -> 缩放和旋转 -> 平移
    total_mat = trans_mat @ rot_mat_3x3 @ flip_mat  # 矩阵乘法
    
    # 从 3x3 矩阵中取出前两行，得到 2x3 的仿射变换矩阵
    final_mat = total_mat[:2, :]
    
    # 使用 cv2.warpAffine 应用变换
    transformed_image = cv2.warpAffine(image, final_mat, (image.shape[1], image.shape[0]), borderValue=(255,255,255))
    
    return transformed_image


# Gradio Interface
def interactive_transform():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Transformation Playground")
        
        # Define the layout
        with gr.Row():
            # Left: Image input and sliders
            with gr.Column():
                image_input = gr.Image(type="pil", label="Upload Image")

                scale = gr.Slider(minimum=0.1, maximum=2.0, step=0.1, value=1.0, label="Scale")
                rotation = gr.Slider(minimum=-180, maximum=180, step=1, value=0, label="Rotation (degrees)")
                translation_x = gr.Slider(minimum=-300, maximum=300, step=10, value=0, label="Translation X")
                translation_y = gr.Slider(minimum=-300, maximum=300, step=10, value=0, label="Translation Y")
                flip_horizontal = gr.Checkbox(label="Flip Horizontal")
            
            # Right: Output image
            image_output = gr.Image(label="Transformed Image")
        
        # Automatically update the output when any slider or checkbox is changed
        inputs = [
            image_input, scale, rotation, 
            translation_x, translation_y, 
            flip_horizontal
        ]

        # Link inputs to the transformation function
        image_input.change(apply_transform, inputs, image_output)
        scale.change(apply_transform, inputs, image_output)
        rotation.change(apply_transform, inputs, image_output)
        translation_x.change(apply_transform, inputs, image_output)
        translation_y.change(apply_transform, inputs, image_output)
        flip_horizontal.change(apply_transform, inputs, image_output)

    return demo

# Launch the Gradio interface
interactive_transform().launch()

In [ ]:
import cv2
import numpy as np
import gradio as gr
from scipy.spatial.distance import cdist 


# 初始化全局变量，存储控制点和目标点
points_src = []
points_dst = []
image = None

# 上传图像时清空控制点和目标点
def upload_image(img):
    global image, points_src, points_dst
    points_src.clear()  # 清空控制点
    points_dst.clear()  # 清空目标点
    image = img
    return img

# 记录点击点事件，并标记点在图像上，同时在成对的点间画箭头
def record_points(evt: gr.SelectData):
    global points_src, points_dst, image
    x, y = evt.index[0], evt.index[1]  # 获取点击的坐标
    
    # 判断奇偶次来分别记录控制点和目标点
    if len(points_src) == len(points_dst):
        points_src.append([x, y])  # 奇数次点击为控制点
    else:
        points_dst.append([x, y])  # 偶数次点击为目标点
    
    # 在图像上标记点（蓝色：控制点，红色：目标点），并画箭头
    marked_image = image.copy()
    for pt in points_src:
        cv2.circle(marked_image, tuple(pt), 1, (255, 0, 0), -1)  # 蓝色表示控制点
    for pt in points_dst:
        cv2.circle(marked_image, tuple(pt), 1, (0, 0, 255), -1)  # 红色表示目标点
    
    # 画出箭头，表示从控制点到目标点的映射
    for i in range(min(len(points_src), len(points_dst))):
        cv2.arrowedLine(marked_image, tuple(points_src[i]), tuple(points_dst[i]), (0, 255, 0), 1)  # 绿色箭头表示映射
    
    return marked_image

# 执行仿射变换

def point_guided_deformation(image, source_pts, target_pts):
    # 获取图像的高度和宽度
    height, width = image.shape[:2]
    
    # 确保控制点为 numpy 数组和浮点类型
    source_pts = np.array(source_pts, dtype=np.float32)
    target_pts = np.array(target_pts, dtype=np.float32)
    
    # 创建网格坐标，并转换为浮点类型
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x.astype(np.float32)
    y = y.astype(np.float32)
    
    # 计算控制点的位移
    displacements = target_pts - source_pts  # 形状：(N, 2)
    
    # 使用 RBF（薄板样条）来插值位移场
    from scipy.interpolate import Rbf
    
    # 提取源点坐标和位移
    src_x = source_pts[:, 0]
    src_y = source_pts[:, 1]
    disp_x = displacements[:, 0]
    disp_y = displacements[:, 1]
    
    # 创建 RBF 插值函数
    rbf_fx = Rbf(src_x, src_y, disp_x, function='thin_plate')
    rbf_fy = Rbf(src_x, src_y, disp_y, function='thin_plate')
    
    # 计算整个图像的位移场
    disp_map_x = rbf_fx(x, y)
    disp_map_y = rbf_fy(x, y)
    
    # 计算新的映射坐标
    map_x = x + disp_map_x.astype(np.float32)
    map_y = y + disp_map_y.astype(np.float32)
    
    # 确保映射坐标在图像范围内
    map_x = np.clip(map_x, 0, width - 1)
    map_y = np.clip(map_y, 0, height - 1)
    
    # 使用 cv2.remap 进行图像变形
    warped_image = cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_LINEAR)
    
    return warped_image



def run_warping():
    global points_src, points_dst, image ### fetch global variables

    warped_image = point_guided_deformation(image, np.array(points_src), np.array(points_dst))

    return warped_image

# 清除选中点
def clear_points():
    global points_src, points_dst
    points_src.clear()
    points_dst.clear()
    return image  # 返回未标记的原图

# 使用 Gradio 构建界面
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            input_image = gr.Image(source="upload", label="上传图片", interactive=True, width=800, height=200)
            point_select = gr.Image(label="点击选择控制点和目标点", interactive=True, width=800, height=800)
            
        with gr.Column():
            result_image = gr.Image(label="变换结果", width=800, height=400)
    
    # 按钮
    run_button = gr.Button("Run Warping")
    clear_button = gr.Button("Clear Points")  # 添加清除按钮
    
    # 上传图像的交互
    input_image.upload(upload_image, input_image, point_select)
    # 选择点的交互，点选后刷新图像
    point_select.select(record_points, None, point_select)
    # 点击运行 warping 按钮，计算并显示变换后的图像
    run_button.click(run_warping, None, result_image)
    # 点击清除按钮，清空所有已选择的点
    clear_button.click(clear_points, None, point_select)
    
# 启动 Gradio 应用
demo.launch()